# Strategy Comparison

This notebook demonstrates how to compare multiple trading strategies.

## Strategies We'll Compare
1. Buy & Hold SPY (100% stocks)
2. Balanced 60/40 (stocks/bonds)
3. Risk Parity
4. Momentum Strategy

In [ ]:
import sys
import os
from datetime import date

sys.path.insert(0, os.path.abspath('..'))

from stocksimulator.data import load_from_csv
from stocksimulator.core.backtester import Backtester
from stocksimulator.strategies import (
    Balanced6040Strategy,
    RiskParityStrategy,
    MomentumStrategy
)

## Load Data

In [ ]:
# Load SPY (stocks) and TLT (bonds)
spy_data = load_from_csv('sp500_stooq_daily.csv', 'SPY', '../historical_data')

try:
    tlt_data = load_from_csv('tlt_stooq_daily.csv', 'TLT', '../historical_data')
    print("✓ Loaded SPY and TLT data")
    has_tlt = True
except FileNotFoundError:
    print("⚠ TLT data not found - will use SPY only strategies")
    has_tlt = False

# Set backtest period
end_date = spy_data.data[-1].date
start_date = date(end_date.year - 5, end_date.month, end_date.day)
print(f"\nBacktest period: {start_date} to {end_date}")

## Run Backtests

In [ ]:
backtester = Backtester(initial_cash=100000.0, transaction_cost_bps=2.0)
results = {}

# Strategy 1: Buy & Hold SPY
def buy_hold(current_date, market_data, portfolio, current_prices):
    return {'SPY': 100.0}

print("Running Buy & Hold...")
results['Buy & Hold'] = backtester.run_backtest(
    'Buy & Hold',
    {'SPY': spy_data},
    buy_hold,
    start_date,
    end_date
)

if has_tlt:
    # Strategy 2: 60/40
    print("Running 60/40...")
    balanced = Balanced6040Strategy()
    results['60/40'] = backtester.run_backtest(
        '60/40',
        {'SPY': spy_data, 'TLT': tlt_data},
        balanced,
        start_date,
        end_date
    )
    
    # Strategy 3: Risk Parity
    print("Running Risk Parity...")
    risk_parity = RiskParityStrategy(lookback_days=60)
    results['Risk Parity'] = backtester.run_backtest(
        'Risk Parity',
        {'SPY': spy_data, 'TLT': tlt_data},
        risk_parity,
        start_date,
        end_date
    )

# Strategy 4: Momentum
print("Running Momentum...")
momentum = MomentumStrategy(lookback_days=126)
results['Momentum'] = backtester.run_backtest(
    'Momentum',
    {'SPY': spy_data},
    momentum,
    start_date,
    end_date
)

print("\n✓ All backtests complete!")

## Compare Results

In [ ]:
import pandas as pd

# Create comparison table
comparison = []

for name, result in results.items():
    summary = result.get_performance_summary()
    comparison.append({
        'Strategy': name,
        'Total Return (%)': f"{summary['total_return']:.2f}",
        'Ann. Return (%)': f"{summary['annualized_return']:.2f}",
        'Sharpe Ratio': f"{summary['sharpe_ratio']:.3f}",
        'Max DD (%)': f"{summary['max_drawdown']:.2f}",
        'Volatility (%)': f"{summary['volatility']:.2f}",
        'Trades': summary['num_trades']
    })

df = pd.DataFrame(comparison)
print("\nSTRATEGY COMPARISON")
print("=" * 80)
print(df.to_string(index=False))

## Visualize Equity Curves

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(14, 8))
    
    for name, result in results.items():
        dates = [point.date for point in result.equity_curve]
        values = [point.value for point in result.equity_curve]
        plt.plot(dates, values, label=name, linewidth=2)
    
    plt.title('Strategy Comparison - Equity Curves', fontsize=16)
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Install matplotlib to visualize: pip install matplotlib")

## Risk-Adjusted Rankings

Let's rank strategies by Sharpe ratio (risk-adjusted return).

In [ ]:
# Sort by Sharpe ratio
sharpe_ranking = []
for name, result in results.items():
    summary = result.get_performance_summary()
    sharpe_ranking.append((name, summary['sharpe_ratio']))

sharpe_ranking.sort(key=lambda x: x[1], reverse=True)

print("\nRISK-ADJUSTED RANKING (by Sharpe Ratio)")
print("=" * 50)
for i, (name, sharpe) in enumerate(sharpe_ranking, 1):
    print(f"{i}. {name:<20} Sharpe: {sharpe:.3f}")

## Key Insights

Compare:
- **Total Return**: Which strategy made the most money?
- **Sharpe Ratio**: Which strategy had the best risk-adjusted return?
- **Max Drawdown**: Which strategy had the smallest peak-to-trough decline?
- **Volatility**: Which strategy was most stable?

Remember: Past performance doesn't guarantee future results!